In [1]:
from dotenv import load_dotenv
import os
import nest_asyncio
nest_asyncio.apply()

load_dotenv()
api_key=os.getenv("MISTRAL_API_KEY")

In [ ]:
#Importing necessary library
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext, load_index_from_storage
from llama_index.embeddings.mistralai import MistralAIEmbedding
from llama_index.llms.mistralai import MistralAI
from llama_index.core import Settings
from IPython.display import Markdown

#Setting up model
embed_model= MistralAIEmbedding(api_key=api_key)
llm= MistralAI(api_key=api_key)
Settings.llm= llm
Settings.embed_model=embed_model

In [3]:
#Reading file name
files=[file for file in os.listdir("../data/ict-policy/processed") if file.endswith('.txt')]

var_names=[]
#Checking the results
for i, file in enumerate(files):
    var_name=file.split('.')[0]
    var_names.append(var_name)
    print(var_name)

MAS
RBI
BB


In [4]:
#Reading documents
docs= {}
for doc in var_names:
    docs[doc]=SimpleDirectoryReader(input_files=["../data/ict-policy/processed/{}.txt".format(doc)]).load_data()

In [5]:
#Checking documents
for doc in var_names:
    print(doc)
    print(docs[doc][0].text[:100])

MAS
# 1 Preface

1.1 The technology landscape of the financial sector is transforming at a rapid pace an
RBI
# Chapter I - Preliminary

1 Short Title and Commencement

(a) These Directions shall be called the 
BB
# Preface

The technology landscape of the financial sector is changing at a rapid pace and the unde


In [ ]:
import re

def split_by_chapters(text):
    # Step 1: Split text into chapters by `#`
    chapters = re.split(r"(# .+)", text)  # Match chapter titles starting with `# `
    chapter_chunks = []

    for i in range(1, len(chapters), 2):  # Iterate over matched chapter titles and their content
        chapter_title = chapters[i].strip()
        chapter_content = chapters[i + 1].strip()
        chapter_chunks.append({
            "title": chapter_title,
            "content": chapter_content
        })


    return chapter_chunks

In [7]:
# Split the text into chapterwise chunks
chapter_chunks= {}
for doc in var_names:
    chapter_chunks[doc] = split_by_chapters(docs[doc][0].text)

In [8]:
#Checking chunk of all docs
for doc in var_names:
    for i, chunk in enumerate(chapter_chunks[doc][:2]):
        print(f"{doc}_Chunk: {i+1}")
        print(chunk)

MAS_Chunk: 1
{'title': '# 1 Preface', 'content': '1.1 The technology landscape of the financial sector is transforming at a rapid pace and the underlying information technology (IT) infrastructure supporting financial services has grown in scope and complexity in recent years. Many financial institutions (FIs) are riding the wave of digitalisation to increase operational efficiency and to deliver better services to consumers.\n\n1.2 Digital transformation in the financial sector can be broadly characterised by the adoption of new technology and the use of existing technology in innovative ways to achieve greater automation and enrich financial service offerings.\n\n1.3 While digital transformation brings significant benefits to the financial ecosystem, it also increases FIs’ exposure to a range of technology risks, including cyber risk. The techniques used by cyber threat actors are becoming increasingly sophisticated, and weak links in the interconnected financial ecosystem can be com

In [9]:
#Importing library for splitting
from llama_index.core import Document
import nltk

def sentence_split(chapter_chunks):
    nodes= []
    for section in chapter_chunks:
        # Preprocess the text to replace multiple consecutive newline characters with a single newline character
        content = re.sub(r'\n{2,}', '\n', section["content"])
        # Preprocess the text to replace newline characters with a space
        content = re.sub(r'(?<!\.)\n', '.\n', content)
        sentences=nltk.sent_tokenize(content)
        for sentence in sentences:
            node= Document(text=sentence,
                        metadata={
                            "title": section["title"]
                        })
            nodes.append(node)
    return nodes

In [10]:
# Splitting chapter_chunks into sentences
sentence_docs= {}
for doc in var_names:
    sentence_docs[doc]=sentence_split(chapter_chunks[doc])

In [11]:
#Cecking sentence from each docs
for doc in var_names:
    for i in range(len(sentence_docs[doc][:2])):
        print(f"\n{doc}_Sentence {i}:")
        print(sentence_docs[doc][i].get_content())
        #print("Title: {}".format(nodes[i].metadata.get("title")))
        #print("Window:")
        #print(nodes[i].metadata.get("window"))


MAS_Sentence 0:
1.1 The technology landscape of the financial sector is transforming at a rapid pace and the underlying information technology (IT) infrastructure supporting financial services has grown in scope and complexity in recent years.

MAS_Sentence 1:
Many financial institutions (FIs) are riding the wave of digitalisation to increase operational efficiency and to deliver better services to consumers.

RBI_Sentence 0:
1 Short Title and Commencement.

RBI_Sentence 1:
(a) These Directions shall be called the Reserve Bank of India (Information Technology Governance, Risk, Controls and Assurance Practices) Directions, 2023.

BB_Sentence 0:
The technology landscape of the financial sector is changing at a rapid pace and the underlying information technology (IT) infrastructure supporting financial services has grown in size and complexity in recent years.

BB_Sentence 1:
Digital transformation in the financial sector can be characterized by the introduction of new technologies and 

In [12]:
#Building index
index= {}
index_directory='../data/ict-policy/index'
for doc in var_names:
    if not os.path.exists(index_directory+f'/{doc}'):
        index[doc]=VectorStoreIndex.from_documents(sentence_docs[doc])
        index[doc].storage_context.persist(persist_dir=index_directory+f'/{doc}')
    else:
        index[doc]= load_index_from_storage(StorageContext.from_defaults(persist_dir=index_directory+f'/{doc}'))

In [13]:
from typing import List
from llama_index.core.indices.vector_store.retrievers import VectorIndexRetriever
from llama_index.core.schema import NodeWithScore, QueryBundle

class AllNodesUniqueTitleVectorIndexRetriever(VectorIndexRetriever):
    def __init__(self, *args, **kwargs):
        # Set similarity_top_k to a very large number to retrieve all nodes
        kwargs['similarity_top_k'] = 10000000000
        super().__init__(*args, **kwargs)
        self.chapter_chunks = kwargs.get('chapter_chunks', [])

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        # Call the parent class's _retrieve method to get all the initial results
        initial_results = super()._retrieve(query_bundle)

        # Filter out duplicate documents based on the "title" metadata
        seen_titles = set()
        unique_results = []
        for result in initial_results:
            title = result.node.metadata.get("title", None)
            if title and title not in seen_titles:
                seen_titles.add(title)
                unique_results.append(result)

        # Sort the unique results by score (if available) and select the top 5
        unique_results.sort(key=lambda x: x.score if x.score is not None else float('inf'), reverse=True)
        top_5_unique_results = unique_results[:5]

        # Replace text with window content after retrieval
        top_5_unique_results = self._replace_text_with_window_content(top_5_unique_results)

        return top_5_unique_results

    def _replace_text_with_window_content(self, results: List[NodeWithScore]) -> List[NodeWithScore]:
        for result in results:
            title = result.node.metadata.get("title")
            window_content = next((chunk["content"] for chunk in self.chapter_chunks if chunk["title"] == title), result.node.text)
            result.node.text = window_content
        return results


In [14]:
# Create an instance of the custom retriever
retrievers= {}
for doc in var_names:
    retrievers[doc] = AllNodesUniqueTitleVectorIndexRetriever(index[doc], chapter_chunks=chapter_chunks[doc])

In [15]:
#Creating query engine
from llama_index.core.query_engine.retriever_query_engine import RetrieverQueryEngine

query_engines= {}

for doc in var_names:
    query_engines[doc]= RetrieverQueryEngine.from_args(retrievers[doc])

In [16]:
query_engine_tool= [
    QueryEngineTool(
        query_engine=query_engines['RBI'],
        metadata= ToolMetadata(
            name="rbi_ICT_document",
            description=(
                "Provide ICT guidelines of Reserve Bank of India (RBI)."
                "Use a detailed plain text question as input to the tool."
            )
        )
    ),
    QueryEngineTool(
        query_engine=query_engines['MAS'],
        metadata= ToolMetadata(
            name="mas_ICT_document",
            description=(
                "Provide ICT guidelines of Monetary Authority of Singapore (MAS)."
                "Use a detailed plain text question as input to the tool."
            )
        )
    ),
    QueryEngineTool(
        query_engine=query_engines['BB'],
        metadata= ToolMetadata(
            name="bangladesh_bank_ICT_document",
            description=(
                "Provide ICT guidelines of Bangladesh Bank (BB)."
                "Use a detailed plain text question as input to the tool."
            )
        )
    ),    
]

In [17]:
from llama_index.core.agent import ReActAgent

agent= ReActAgent.from_tools(
    query_engine_tool, verbose=True, allow_parallel_tool_calls=True
)

In [18]:
response= agent.chat(
    "What provisions are made for penetration testing at MAS, RBI and BB. When giving final answer, summarize it with heading for each tool response."
)

> Running step 50c0c905-0e67-494e-b201-c244d8864ac5. Step input: What provisions are made for penetration testing at MAS, RBI and BB. When giving final answer, summarize it with heading for each tool response.
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: rbi_ICT_document
Action Input: {'input': 'What provisions are made for penetration testing?'}
Observation: Penetration testing should be conducted at least annually for critical information systems and those in the De-Militarized Zone (DMZ) with customer interfaces. For non-critical systems, the need and frequency of penetration testing should be determined based on a risk assessment. These tests should be performed by independent and trained information security experts or auditors. In post-implementation scenarios, penetration testing should ideally be done in the production environment, but if conducted in a test environment, it must closely resemble the producti

In [19]:
display(Markdown(response.response))

### Reserve Bank of India (RBI)

Penetration testing should be conducted at least annually for critical information systems and those in the De-Militarized Zone (DMZ) with customer interfaces. For non-critical systems, the need and frequency of penetration testing should be determined based on a risk assessment. These tests should be performed by independent and trained information security experts or auditors. In post-implementation scenarios, penetration testing should ideally be done in the production environment, but if conducted in a test environment, it must closely resemble the production environment. Any deviations should be documented and approved by the Information Security Committee (ISC). Identified vulnerabilities should be addressed promptly to prevent recurrence. Additionally, a documented approach should be in place outlining the scope, coverage, vulnerability scoring mechanism, and other aspects of penetration testing.

### Monetary Authority of Singapore (MAS)

Penetration testing should be conducted to evaluate the effectiveness of cyber security defenses. This includes a combination of blackbox testing, where the tester has no prior knowledge of the environment, and greybox testing, where the tester has credentials and authenticated access. For online financial services, both methods should be employed. Additionally, a bug bounty program can be considered to incentivize ethical hackers to identify vulnerabilities.

Penetration testing should be performed on the production environment with proper safeguards in place. The frequency of penetration testing should be determined based on the system's criticality and exposure to cyber risks. For systems accessible from the Internet, penetration testing should be conducted at least annually or after major changes or updates.

### Bangladesh Bank (BB)

Penetration testing should be conducted at least annually and after any significant infrastructure or application upgrade or modification. The methodology for penetration testing should be based on industry-accepted approaches and include coverage for critical systems, testing from both inside and outside the network, a review of threats and vulnerabilities experienced in the last 12 months, and the retention of penetration testing results and remediation results. Additionally, internal vulnerability scans should be performed at least twice a year, while external vulnerability scans for critical systems should be conducted annually by an independent party.

In [20]:
response= agent.chat(
    """What provisions are made for IT governance at MAS, RBI and BB. When giving final answer, summarize it with heading for each tool response."""
)

> Running step 5a415d2c-b584-4c80-9258-1fb505863321. Step input: What provisions are made for IT governance at MAS, RBI and BB. When giving final answer, summarize it with heading for each tool response.
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: rbi_ICT_document
Action Input: {'input': 'What provisions are made for IT governance at RBI?'}
Observation: The provisions for IT governance include establishing a robust IT Governance Framework that focuses on strategic alignment, risk management, resource management, performance management, and Business Continuity/Disaster Recovery Management. This framework specifies the governance structure and processes to meet business objectives, defines roles and responsibilities of the Board and Senior Management, and includes oversight mechanisms for accountability and risk mitigation. Additionally, enterprise-wide risk management policies must incorporate periodic assessments o

In [21]:
display(Markdown(response.response))

### Reserve Bank of India (RBI)

The provisions for IT governance include establishing a robust IT Governance Framework that focuses on strategic alignment, risk management, resource management, performance management, and Business Continuity/Disaster Recovery Management. This framework specifies the governance structure and processes to meet business objectives, defines roles and responsibilities of the Board and Senior Management, and includes oversight mechanisms for accountability and risk mitigation. Additionally, enterprise-wide risk management policies must incorporate periodic assessments of IT-related risks. The Board of Directors is responsible for approving and reviewing IT strategies and policies annually, while an IT Strategy Committee (ITSC) is established to oversee IT strategic planning and risk management. Senior Management ensures the execution of IT strategies and maintains effective IT risk management processes. An IT Steering Committee is also established to assist the ITSC and oversee IT performance and business continuity processes. The Head of IT Function is responsible for executing IT projects, maintaining an effective IT structure, and managing IT controls and risks.

### Monetary Authority of Singapore (MAS)

The provisions for IT governance include establishing a sound and robust technology risk management framework, overseen by the board of directors and senior management. This involves creating a technology risk management strategy, appointing key roles such as a Chief Information Officer and a Chief Information Security Officer, and ensuring that technology risks are managed in accordance with the FI's risk appetite. Additionally, a strong culture of technology risk awareness and management should be cultivated, and a technology risk management function should be established to oversee the framework and provide an independent view of technology risks. Regular reviews and assessments of the technology risk management strategy and management competencies are also required.

### Bangladesh Bank (BB)

Several provisions are made for IT governance. These include ensuring that IT functions and operations are efficiently managed, with appropriate IT security controls in place. Top management is responsible for contributing to IT security planning to align resources with business objectives and to employ qualified technical staff. IT Security Management oversees various aspects, including roles and responsibilities, IT security policy, documentation, internal and external information system audits, training and awareness, insurance, and risk coverage fund.

Key stakeholders in IT governance include the Board, IT Steering Committee, IT Security Committee, CEO/Managing Director, CIO, CTO, CITO, CISO, Risk Management Committee,

In [22]:
response= agent.chat(
    """What role of board is envisioned for IT governance at MAS, RBI and BB. When giving final answer, summarize it with heading for each tool response."""
)

> Running step 4c4addb1-7811-4589-bfb7-0872484791d7. Step input: What role of board is envisioned for IT governance at MAS, RBI and BB. When giving final answer, summarize it with heading for each tool response.
Thought: (Implicit) I can answer without any more tools!
Answer: ### Reserve Bank of India (RBI)

The Board of Directors plays a crucial role in IT governance by approving and reviewing IT strategies and policies annually. They are responsible for ensuring that the IT Governance Framework aligns with business objectives and effectively manages IT-related risks. The Board also oversees the establishment of an IT Strategy Committee (ITSC) to focus on IT strategic planning and risk management. Their role includes ensuring that the IT governance structure is robust and that accountability mechanisms are in place.

### Monetary Authority of Singapore (MAS)

The Board of Directors is responsible for overseeing the technology risk management framework, ensuring that it aligns with the

In [23]:
display(Markdown(response.response))

### Reserve Bank of India (RBI)

The Board of Directors plays a crucial role in IT governance by approving and reviewing IT strategies and policies annually. They are responsible for ensuring that the IT Governance Framework aligns with business objectives and effectively manages IT-related risks. The Board also oversees the establishment of an IT Strategy Committee (ITSC) to focus on IT strategic planning and risk management. Their role includes ensuring that the IT governance structure is robust and that accountability mechanisms are in place.

### Monetary Authority of Singapore (MAS)

The Board of Directors is responsible for overseeing the technology risk management framework, ensuring that it aligns with the FI's risk appetite and overall business strategy. The Board appoints key roles such as the Chief Information Officer (CIO) and Chief Information Security Officer (CISO) and ensures that technology risks are effectively managed. They are also responsible for cultivating a strong culture of technology risk awareness and management within the organization.

### Bangladesh Bank (BB)

The Board of Directors plays a significant role in IT governance by ensuring that IT functions and operations are efficiently managed with appropriate IT security controls in place. The Board contributes to IT security planning to align resources with business objectives and ensures the employment of qualified technical staff. They oversee the IT Steering Committee, IT Security Committee, and other key stakeholders to ensure effective IT governance and risk management.

In [24]:
response= agent.chat(
    """What role of board is envisioned for IT governance at MAS, RBI. When giving final answer, summarize it with heading for each tool response."""
)

> Running step e4192f7e-c8b4-4d3c-9bc3-3064a6c6789f. Step input: What role of board is envisioned for IT governance at MAS, RBI. When giving final answer, summarize it with heading for each tool response.
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: rbi_ICT_document
Action Input: {'input': 'What role of board is envisioned for IT governance at RBI?'}
Observation: The Board of Directors is responsible for approving and reviewing strategies and policies related to IT, Information Assets, Business Continuity, Information Security, and Cyber Security, including Incident Response and Recovery Management. These strategies and policies should be reviewed by the Board at least annually. Additionally, the Board oversees the establishment of an IT Strategy Committee to ensure effective IT governance and alignment of IT strategy with the organization's business objectives. The Board also ensures that the IT governance structur

In [25]:
display(Markdown(response.response))

### Reserve Bank of India (RBI)

The Board of Directors is responsible for approving and reviewing strategies and policies related to IT, Information Assets, Business Continuity, Information Security, and Cyber Security, including Incident Response and Recovery Management. These strategies and policies should be reviewed by the Board at least annually. Additionally, the Board oversees the establishment of an IT Strategy Committee to ensure effective IT governance and alignment of IT strategy with the organization's business objectives. The Board also ensures that the IT governance structure fosters accountability and includes adequate oversight mechanisms to mitigate IT and cybersecurity risks.

### Monetary Authority of Singapore (MAS)

The board of directors is envisioned to play a crucial role in IT governance at MAS. They are responsible for ensuring a sound and robust risk management framework is established and maintained to manage technology risks. This includes approving the risk appetite and risk tolerance statement, undertaking regular reviews of the technology risk management strategy, and ensuring an independent audit function is established. Additionally, the board should ensure that senior executives responsible for executing the FI’s technology risk management strategy have sufficient authority, resources, and access to the board. They should also assess management competencies for managing technology risks and ensure a technology risk management function is in place to oversee the framework and strategy.